In [9]:
#!pip install neo4j                                # uncomment the pip commands if running in google colab
#!pip install python-dotenv

from neo4j import GraphDatabase
import dotenv
import os
import numpy as np
import pandas as pd

###  Ensure that data url is neo4j+ssc and not neo4j+s as it gives error. Replace URI with your URI and upload config file of neo4j in Data folder

In [10]:
load_status = dotenv.load_dotenv("../Data/Neo4j-71bfe867-Created-2024-03-27.txt")       # replace by your neo4j AuraDB credential txt file
if load_status is False:
    raise RuntimeError('Environment variables not loaded.')

URI =  "neo4j+ssc://71bfe867.databases.neo4j.io"            #here use neo4j+ssc
AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
print(URI,AUTH)
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

neo4j+ssc://71bfe867.databases.neo4j.io ('neo4j', '4gq3gmEVzj_dfp5ihwePYByBoj22fqrY7aJhrEPJ9cQ')


#### Dummy cypher query. Works only if neo4j DB already initialized


In [11]:
query="MATCH (n:Product) RETURN n LIMIT 25;"            # dummy query to show that it is connected to database
summary=driver.execute_query(query)
summary

<ipython-input-11-83817907dfc2>:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  summary=driver.execute_query(query)


EagerResult(records=[<Record n=<Node element_id='4:6380264d-7352-4e5c-8881-c089509078c6:0' labels=frozenset({'Product'}) properties={'review_cnt': 2, 'downloads': 2, 'rating': 5.0, 'id': 1, 'title': 'Patterns of Preaching: A Sermon Sampler', 'salesrank': 396585, 'group': 'Book'}>>, <Record n=<Node element_id='4:6380264d-7352-4e5c-8881-c089509078c6:1' labels=frozenset({'Product'}) properties={'review_cnt': 12, 'downloads': 12, 'rating': 4.5, 'id': 2, 'title': 'Candlemas: Feast of Flames', 'salesrank': 168596, 'group': 'Book'}>>, <Record n=<Node element_id='4:6380264d-7352-4e5c-8881-c089509078c6:2' labels=frozenset({'Product'}) properties={'review_cnt': 1, 'downloads': 1, 'rating': 5.0, 'id': 3, 'title': 'World War II Allied Fighter Planes Trading Cards', 'salesrank': 1270652, 'group': 'Book'}>>, <Record n=<Node element_id='4:6380264d-7352-4e5c-8881-c089509078c6:3' labels=frozenset({'Product'}) properties={'review_cnt': 1, 'downloads': 1, 'rating': 4.0, 'id': 4, 'title': 'Life Applicatio

In [24]:
products = pd.read_csv('../Data/products.csv')
copurchase = pd.read_csv('../Data/copurchase.csv')

In [25]:
products.head()

,id,title,group,salesrank,review_cnt,downloads,rating
0,1,Patterns of Preaching: A Sermon Sampler,Book,396585.0,2,2,5.0
1,2,Candlemas: Feast of Flames,Book,168596.0,12,12,4.5
2,3,World War II Allied Fighter Planes Trading Cards,Book,1270652.0,1,1,5.0
3,4,Life Application Bible Commentary: 1 and 2 Tim...,Book,631289.0,1,1,4.0
4,5,Prayers That Avail Much for Business: Executive,Book,455160.0,0,0,0.0


In [26]:
products=products.values                                        #Converting to Numpy
copurchase=copurchase.values

### Following code block removes all relationships to products having ID greater than 87000. The purpose for doing this is to ensure there are less than 400k relationships only

In [27]:
max_count=0
max_row=87000
copurchase=copurchase[:417526 + 1]
arr=[]
print(f"Co-purchase len : {len(copurchase)}")
for i in range(len(copurchase)):
        row=copurchase[i]
        second=row[1]
        if second>max_row:
          arr.append(i)

for i in range(len(arr)):
  copurchase=np.delete(copurchase,arr[i]-i,axis=0)


copurchase=copurchase[:-2]
print(len(copurchase))

Co-purchase len : 417527
385007


In [31]:
arr=[]
for i in range(len(copurchase)):                  ## Checks if there are any remaining relation with a node with ID>87000
  second=copurchase[i][1]
  if second>87000:
    arr.append(i)
print(arr)
for num in range(len(arr)):
  copurchase=np.delete(copurchase,i,axis=0)



[]


In [32]:
column_names = ['Source', 'Target']

In [33]:
copurchase_new=pd.DataFrame(data=copurchase, columns=column_names)

In [34]:
copurchase_new.tail()

,Source,Target
385000,86999,72804
385001,87000,19409
385002,87000,71044
385003,87000,73750
385004,87000,73751


In [37]:
# prompt: how to create csv file with pandas dataframe and save it

copurchase_new.to_csv('../Data/copurchase_new.csv', index=False)


In [38]:
products=products[:87002]

In [39]:
products[-1]

array([87929, 'Baton Rouge', 'Music', 154280.0, 4, 4, 5.0], dtype=object)

In [40]:
index=0
for i in range(len(products)):
  if products[i][0]==87000:
    index=i
print(index)

86086


In [43]:
products=products[:86087]

In [44]:
column_arr=["id","title","group","salesrank","review_cnt","downloads","rating"]

In [46]:
products=pd.DataFrame(data=products, columns=column_arr)

In [48]:
products.to_csv('../Data/product_new.csv', index=False)
